In [1]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from sklearn import preprocessing
import numpy as np
import random
random.seed(42)

In [2]:
PATH='../../data/GoogleAnalyticsCustomerRevenuePrediction/'

In [3]:
%%time
tr1 = pd.read_pickle(f'{PATH}tr1_clean')
tr2 = pd.read_pickle(f'{PATH}tr2_clean')
tr3 = pd.read_pickle(f'{PATH}tr3_clean')
tr4 = pd.read_pickle(f'{PATH}tr4_clean')

CPU times: user 1.44 s, sys: 584 ms, total: 2.02 s
Wall time: 2.4 s


In [4]:
tr1.shape, tr2.shape, tr3.shape, tr4.shape,

((377186, 42), (288869, 42), (385318, 42), (366202, 42))

In [5]:
train = pd.concat([tr1, tr2, tr3, tr4], axis=0, sort=False).reset_index(drop=True)

In [6]:
# correct data types
train['interval_dates'] = train['interval_dates'].dt.days
train['first_ses_from_the_period_start'] = train['first_ses_from_the_period_start'].dt.days
train['last_ses_from_the_period_end'] = train['last_ses_from_the_period_end'].dt.days

In [7]:
train_data_X = train.drop(['target', 'ret'], axis=1)
train_data_y = train['target']
train_data_ret = train['ret']

In [8]:
num_cols = [colname for colname in train_data_X.columns if train_data_X[colname].dtype!= 'object']
cat_cols = set(train_data_X.columns) - set(num_cols)

In [9]:
# fillmissing 
for col in cat_cols:
    train_data_X[col].fillna('missing', inplace=True) 

for col in num_cols:       
    train_data_X[col].fillna(0, inplace=True)

In [10]:
# label encoding
les = []
for col in cat_cols:
    le = preprocessing.LabelEncoder()
    le.fit(train_data_X[col])
    les.append(le)
    train_data_X[col] = le.transform(train_data_X[col])    

In [11]:
with open('les.pickle', 'wb') as f:
    pickle.dump(les, f, -1)

In [12]:
# standardize the num_cols

# Create the Scaler object
scaler = preprocessing.StandardScaler()

# Fit your data on the scaler object
scaled_df = scaler.fit_transform(train_data_X[num_cols])
scaled_df = pd.DataFrame(scaled_df, columns=num_cols)
train_data_X[num_cols] = scaled_df

/home/nick/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/nick/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [13]:
train_data_y = np.array(train_data_y)
train_data_ret = np.array(train_data_ret)

with open('feature_train_data_v2.pickle', 'wb') as f:
    pickle.dump((train_data_X, train_data_y, train_data_ret), f, -1)   

In [14]:
with open('col_types.pickle', 'wb') as f:
    pickle.dump((cat_cols, num_cols), f, -1) 